# Face Quality Assessment for Face Verification in Video 
https://pdfs.semanticscholar.org/2c0a/caec54ab2585ff807e18b6b9550c44651eab.pdf?_ga=2.118968650.2116578973.1552199994-98267093.1547624592

In [177]:
pwd

'/home/vision/pose/lfw-youtube-experiments_v2'

In [115]:
import tensorflow as tf
from detection.mtcnn import detect_face
with tf.Graph().as_default():
    sess = tf.Session()
    pnet, rnet, onet = detect_face.create_mtcnn(sess, None)

In [116]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import line_profiler 
%load_ext line_profiler

img = cv2.imread('alireza.jpg')
gray = cv2.imread('alireza.jpg', 0)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [117]:
minsize = 20 # minimum size of face
threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
factor = 0.709 # scale factor
bounding_boxes, points = detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)

# Illumination 

In [118]:
def illumination(gray):
    # length of R available  range  of  gray  intensities  excluding  5%  of  the darkest  and  brightest  pixel
    sorted_gray = np.sort(gray.ravel())
    l = len(sorted_gray)
    cut_off_idx = l * 5 // 100
    r = sorted_gray[l-cut_off_idx] - sorted_gray[cut_off_idx]
    return np.round(r / 255, 2)

In [119]:
illumination(gray)

0.86

# Sharpness

In [128]:
def get_contour(pts):
    return np.array([[pts[i], pts[5 + i]] for i in  [0, 1, 4, 3]], np.int32).reshape((-1,1,2))

def get_mask(image, contour):
    mask = np.zeros(image.shape[0:2],dtype="uint8")
    cv2.drawContours(mask, [contour], -1, 255, -1)
    return mask

In [151]:
def laplacian_over_mask(image, landmark):
    zero = np.zeros_like(image)
    contour = get_contour(landmark)
    mask = get_mask(image, contour) #one-channel mask
    mask = np.stack((mask,)*3, axis=-1) #3-channel mask
    image = image.astype(np.uint8)
    return cv2.bitwise_and(image, mask)

In [176]:
def sharpness(img, landmark):
    contour = get_contour(landmark)
    mask = get_mask(img, contour) #1-channel mask
    mask = np.stack((mask,)*3, axis=-1) #3-channel mask
    mask[mask == 255] = 1 # convert 0 and 255 to 0 and 1
    laplacian = cv2.Laplacian(img,cv2.CV_64F)
    edges = laplacian[mask.astype(bool)]
    return np.round(edges.var() / 255 , 2)

sharpness(img, points[:,0])

0.92